In [108]:
import json
import yfinance
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs['company_name']
    return ddg.run(f"Ticker symbol of {company_name}")

def get_stock(inputs):
    ticker = inputs['ticker']
    return yfinance.Ticker(ticker)
    

def get_income_statement(inputs):
    '''손익계산서'''
    stock = get_stock(inputs)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    '''대차대조표'''
    stock = get_stock(inputs)
    return json.dumps(stock.balance_sheet.to_json())
    
def get_daily_stock_performance(inputs):
    ''''''
    stock = get_stock(inputs)
    return json.dumps(stock.history(period='3mo').to_json())

functions_map = {
    "get_ticker": get_ticker,
    'get_income_statement': get_income_statement,
    'get_balance_sheet': get_balance_sheet,
    'get_daily_stock_performance': get_daily_stock_performance,
}

functions = [
    {
      "type": "function",
      "function": {
          "name": "get_ticker",
          "description": "Given the name of a company returns its ticker symbol",
          "parameters": {
              "type": "object",
              "properties": {
                  "company_name": {
                      "type": "string",
                      "description": "The name of the company"
                  }
              },
              "required": [
                  "company_name",
              ]
          }
      }
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [109]:
import openai as client

# assistant = openai.beta.assistants.create(
#     name='Investor Assistant',
#     instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.',
#     model='gpt-4-turbo-2024-04-09',
#     tools=functions,
# )

assistant_id = "asst_wMlEYSv50bRst7PWaofz6xBE"

In [110]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy"
        }
    ]
)
thread

Thread(id='thread_Cq7N3vvfpbIvovRjgTuqz0t4', created_at=1713013448, metadata={}, object='thread')

In [111]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_kRGtPjDDIrZ36gHAhqbScknl', assistant_id='asst_wMlEYSv50bRst7PWaofz6xBE', cancelled_at=None, completed_at=None, created_at=1713013449, expires_at=1713014049, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-turbo-2024-04-09', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_Cq7N3vvfpbIvovRjgTuqz0t4', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'ty

In [112]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(run_id=run_id, thread_id=thread_id)

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id).data
    messages.reverse()
    for message in messages:
        print(f"{message.role}: ", end=' ')
        if message.content:
            print(message.content[0].text.value)

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append({
            "output": functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id": action_id,
        })
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [113]:
get_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [114]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Salesforce"}


Run(id='run_kRGtPjDDIrZ36gHAhqbScknl', assistant_id='asst_wMlEYSv50bRst7PWaofz6xBE', cancelled_at=None, completed_at=None, created_at=1713013449, expires_at=1713014049, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-turbo-2024-04-09', object='thread.run', required_action=None, started_at=1713013449, status='queued', thread_id='thread_Cq7N3vvfpbIvovRjgTuqz0t4', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameter

In [115]:
get_run(run.id, thread.id).status

'requires_action'

In [116]:
get_messages(thread.id)

user:  I want to know if the Salesforce stock is a good buy


In [117]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_kRGtPjDDIrZ36gHAhqbScknl', assistant_id='asst_wMlEYSv50bRst7PWaofz6xBE', cancelled_at=None, completed_at=None, created_at=1713013449, expires_at=1713014049, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-turbo-2024-04-09', object='thread.run', required_action=None, started_at=1713013471, status='queued', thread_id='thread_Cq7N3vvfpbIvovRjgTuqz0t4', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameter

In [118]:
get_messages(thread.id)

user:  I want to know if the Salesforce stock is a good buy
assistant:  

In [119]:
get_messages(thread.id)

user:  I want to know if the Salesforce stock is a good buy
assistant:  

In [120]:
send_message(thread.id, "Go ahead!")

Message(id='msg_HgQM8tw7VfGFuXZTUNGqzAXs', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Go ahead!'), type='text')], created_at=1713013509, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Cq7N3vvfpbIvovRjgTuqz0t4')

In [121]:
get_messages(thread.id)

user:  I want to know if the Salesforce stock is a good buy
assistant:  ### Salesforce (Ticker: CRM) Financial Overview

#### Income Statement Highlights (Latest Fiscal Year)
- **Total Revenue:** $34.86 billion
- **Gross Profit:** $26.32 billion
- **Operating Income:** $5.99 billion
- **Net Income:** $4.14 billion
- **EBITDA:** $9.96 billion
- **EPS (Diluted):** $4.20

#### Balance Sheet Highlights
- **Total Assets:** $99.82 billion
- **Total Liabilities:** $40.18 billion
- **Total Equity:** $59.65 billion
- **Total Debt:** $12.59 billion
- **Working Capital:** $2.44 billion

#### Stock Performance (Last 100 Days)
- The stock price has shown some volatility but also periods of growth, currently trending upwards in the latest data. The trading volume has remained healthy and active.

### Analysis & Insights
1. **Strong Revenue & Profitability:** Salesforce has showcased strong revenue generation capabilities along with a robust gross profit. This reflects healthy business practices and 

In [5]:
import json
import openai as client
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper

def get_wpd_result(inputs):
    wpd = WikipediaAPIWrapper()
    theme = inputs["theme"]
    return wpd.run(theme)


def get_ddg_result(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    theme = inputs["theme"]
    return ddg.run(theme)

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_wpd_result",
            "description": "Returns content searched in Wikipedia with the given theme.",
            "parameters": {
                "type": "object",
                "properties": {
                    "theme": {
                        "type": "string",
                        "description": "The theme that user wants to search",
                    }
                },
                "required": {
                    "theme",
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ddg_result",
            "description": "Returns content searched in DuckDuckGo with the given theme.",
            "parameters": {
                "type": "object",
                "properties": {
                    "theme": {
                        "type": "string",
                        "description": "The theme that user wants to search",
                    }
                },
                "required": {
                    "theme",
                },
            },
        },
    },
]
assistant = client.beta.assistants.create(
    name='Theme Research Assistant',
    instructions='You are an assistant who searches duckduckgo and wikipedia for the topics the user wants, compiles them, and informs them.',
    model='gpt-4-turbo-2024-04-09',
    tools=functions,
)


TypeError: Object of type set is not JSON serializable